# ZTF alerts triggered in TESS Northern Fields

The [Zwicky Transient Facility (ZTF)](https://ztf.caltech.edu) will conduct a public survey of all 13 TESS northern sectors in 2019-2020. The sectors (as much as is visible from the Palomar Observatory on a particular night) will be mapped every night in the $g$ and $r$ filters concurrently with the TESS observations. 

The observations of the first TESS Northern sector (sector 14) have begun on July 18, 2019. 

The ZTF alerts for the transient events discovered in the TESS fields will have the [alert packet](https://zwickytransientfacility.github.io/ztf-avro-alert/) field `programpi` in the `candidate` block set to `TESS`. Note that events may be triggered from transient, reoccurring variable, or moving objects.

A compressed nightly tar-ball with the alert packets converted into the `JSON` format will be immediately made available to the public via ZTF's bucket on `Google Cloud`. The alerts in the TESS fields will be additionally cross-matched against a number of external catalogs. Furthermore, we anticipate executing multiple machine learning (ML) classifiers on the alerts. Initially, the catalogs will include `2MASS_PSC`, `AllWISE`, `IPHAS_DR2`, and `Gaia_DR2` and the ML classifiers will include `braai`, a deep-learning-based real-bogus classifier.